## Evaluating a vectorial function on CPU and GPU

### CPU: plain and numpy

In [33]:
import numpy as np
from numba import njit, jit

# Python plain implementation w/ numba 
@njit
def grade2_vector(x, y, a, b, c):
    z = np.zeros(x.size)
    for i in range(x.size):
        z[i] = a*x[i]*x[i] + b*y[i] + c
    return z

# Numpy ufunc
def grade2_ufunc(x, y, a, b, c):
    return a*x**2 + b*y + c

# size of the vectors
size = 5_000_000

# allocating and populating the vectors
a_cpu = np.random.rand(size)
b_cpu = np.random.rand(size)
c_cpu = np.zeros(size)

a = 3.5
b = 2.8
c = 10

# Printing input values
#print(a_cpu)
#print(b_cpu)
# Random function in Numpy always use float64
print(a_cpu.dtype)

c_cpu = grade2_vector(a_cpu, b_cpu, a, b, c)


# Evaluating the time

# Numba Python: huge improvement, better that numpy code
%timeit -n 5 -r 2 grade2_vector(a_cpu, b_cpu, a, b, c)

# w/ a numpy ufunc manually coded
%timeit -n 5 -r 2 grade2_ufunc(a_cpu, b_cpu, a, b, c)

# using the general numpy ufunc 
%timeit -n 5 -r 2 a*a_cpu**2 + b*b_cpu + c



float64
8.57 ms ± 23.5 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
18.2 ms ± 108 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
18 ms ± 51 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)


### a) Librería cupy

#### Caso 1: copiando los arrays entre CPU y GPU

In [34]:
import cupy as cp
from cupyx.profiler import benchmark


# Using cupy
def grade2_ufunc_cupy(x, y, a, b, c):
    # Pasamos los valores de la CPU a GPU
    a_gpu = cp.asarray(a_cpu)
    b_gpu = cp.asarray(b_cpu)
    return a*x**2 + b*y + c



## CARGA DE DATOS
# Tamaño de los vectores
size = 5_000_000

# Cargamos los datos en CPU
a_cpu = np.random.rand(size)
b_cpu = np.random.rand(size)


a = 3.5
b = 2.8
c = 10


## CÁLCULO Y TIEMPO (usando benchmark)
t_copy = benchmark(grade2_ufunc_cupy, 
                   (a_gpu, b_gpu, a, b, c), 
                   n_repeat = 5)

gpu_avg_time = np.average(t_copy.gpu_times) * 1e3  #Pasamos a ms

print(f"Tiempo GPU (datos copiados CPU a GPU): {gpu_avg_time:.3f} ms \n")


Tiempo GPU (datos copiados CPU a GPU): 14.153 ms 



#### Caso 2: datos creados en GPU

In [35]:
import cupy as cp
from cupyx.profiler import benchmark


# Using cupy
def grade2_ufunc_cupy(x, y, a, b, c):
    return a*x**2 + b*y + c


## CARGA DE DATOS
# Tamaño de los vectores
size = 5_000_000

# Creamos directamente los datos en GPU
a_gpu = cp.random.rand(size)
b_gpu = cp.random.rand(size)

a = 3.5
b = 2.8
c = 10


## CÁLCULO Y TIEMPO (usando benchmark)
t_created = benchmark(grade2_ufunc_cupy, 
                   (a_gpu, b_gpu, a, b, c), 
                   n_repeat = 5)

gpu_avg_time = np.average(t_created.gpu_times) * 1e3  #Pasamos a ms

print(f"Tiempo GPU (datos creados en GPU): {gpu_avg_time:.3f} ms \n")


Tiempo GPU (datos creados en GPU): 4.267 ms 



### b) Librería Numba

#### Caso 1: copiando arrays de CPU a GPU

In [48]:
import cupy as cp
from cupyx.profiler import benchmark
from numba import vectorize


# Using cupy
@vectorize(['float64(float64, float64, float64, float64, float64)'],
           target='cuda')
def grade2_ufunc_numba(x, y, a, b, c):
    return a*x**2 + b*y + c



## CARGA DE DATOS
# Tamaño de los vectores
size = 5_000_000

# Cargamos los datos en CPU
a_cpu = np.random.rand(size)
b_cpu = np.random.rand(size)


# Pasamos los valores de la CPU a GPU
# a_gpu = cp.asarray(a_cpu)
# b_gpu = cp.asarray(b_cpu)
# LA TRANSFERENCIA ES AUTOMÁTICA CON NUMBA

a = 3.5
b = 2.8
c = 10


## CÁLCULO Y TIEMPO (usando benchmark)
# En este tiempo se incluye transferencia + ejecución
t_copy = benchmark(grade2_ufunc_numba, 
                   (a_cpu, b_gpu, a, b, c), 
                   n_repeat = 5)

gpu_avg_time = np.average(t_copy.gpu_times) * 1e3  #Pasamos a ms

print(f"Tiempo GPU (datos copiados CPU a GPU): {gpu_avg_time:.3f} ms \n")


Tiempo GPU (datos copiados CPU a GPU): 7.113 ms 



#### Caso 2: creando los datos directamente en GPU

In [49]:
from numba import vectorize
import cupy
from cupyx.profiler import benchmark


# Using numba
@vectorize(['float64(float64, float64, float64, float64, float64)'],
           target='cuda')
def grade2_ufunc_numba(x, y, a, b, c):
    return a*x**2 + b*y + c


## CARGA DE DATOS
# Tamaño de los vectores
size = 5_000_000

# Creamos directamente los datos en GPU
a_gpu = cp.random.rand(size)
b_gpu = cp.random.rand(size)

a = 3.5
b = 2.8
c = 10


## CÁLCULO Y TIEMPO (usando benchmark)
t_created = benchmark(grade2_ufunc_numba, 
                   (a_gpu, b_gpu, a, b, c), 
                   n_repeat = 5)

gpu_avg_time = np.average(t_created.gpu_times) * 1e3  #Pasamos a ms

print(f"Tiempo GPU (datos creados en GPU): {gpu_avg_time:.3f} ms \n")


Tiempo GPU (datos creados en GPU): 2.090 ms 



## c) Interpretación de resultados
### Empleando CuPy
#### Tiempo GPU (datos copiados CPU a GPU): 14.153 ms
#### Tiempo GPU (datos creados en GPU): 4.267 ms
Como podemos ver, cuando se crean los datos en CPU y después se copian a GPU, obtenemos casi 10 ms más de tiempo empleado respecto a cuando los datos son creados directamente en la GPU (esos 10 ms más se corresponderían con el tiempo de transferencia).

Esto se debe al tiempo de carga de los datos, es decir, el tiempo que tardan en pasarse de la CPU a la GPU; mientras que si se crean directamente en GPU (mediante CuPy), nos ahorramos ese tiempo.



### Empleando Numba
#### Tiempo GPU (datos copiados CPU a GPU): 7.113 ms
#### Tiempo GPU (datos creados en GPU): 2.090 ms
Cuando empleamos Numba, obtenemos resultados similares a empleando CuPy: la creación de datos directamente en GPU ayuda a reducir el tiempo empleado al quitarnos de en medio la transferencia de datos de CPU a GPU.

Cuando comparamos **CuPy vs Numba**, vemos una aceleración en los cálculos realizados por Numba, además de que permite compilar ufuncs a medida, mostrando un mejor rendimiento respecto de CuPy.

Así, aunque el cálculo en GPU es muy eficiente, la transferencia de datos puede convertirse en un cuello de botella.